## ライブラリ導入

In [ ]:
from qiskit import *
from qiskit.quantum_info import Operator, Statevector
from qiskit.visualization import *
from qiskit_aer import Aer
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy.optimize import minimize
import commonLogic

%matplotlib inline

## パラメータ設定

In [ ]:
# ノード数
num = 5
# エッジ構造
edges = [(0,1),(0,2),(1,2),(1,3),(2,4),(3,4)]

gamma = 0.1
beta = 0.1

In [ ]:
# コストハミルトニアン
def get_operator_cost(gamma):
    qc_cost = QuantumCircuit(num)

    for i,j in edges:
        qc_cost.cx(i, j)
        qc_cost.rz(-2*gamma, j)
        qc_cost.cx(i, j)

    operator_cost = Operator(qc_cost)
    return operator_cost

In [ ]:
# ミキサーハミルトニアン
def get_operator_mixer(beta):
    qc_mixer = QuantumCircuit(num)

    for i,j in edges:
        qc_mixer.rx(-2*beta, i)

    operator_mixer = Operator(qc_mixer)
    return operator_mixer

## グラフ描画

In [ ]:
G = nx.Graph()
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True)
plt.show()

## 実装

In [ ]:
def get_mainCircuit(num,gamma,beta):
    qc_main = QuantumCircuit(num,num)
    qc_main.h(range(num))
    qc_main.append(get_operator_cost(gamma),range(num))
    qc_main.append(get_operator_mixer(beta),range(num))
    qc_main.measure(range(num),range(num))
    return qc_main

def get_expectation_value(args):
    gamma,beta = args

    qc = get_mainCircuit(num,gamma,beta)
    operator_cost = get_operator_cost(gamma)

    backend = Aer.get_backend("statevector_simulator")
    qc_transpiled = transpile(qc,backend)
    result = backend.run(qc).result()
    statevector = result.get_statevector()
    expectation_value = statevector.expectation_value(operator_cost).real
    return expectation_value

In [ ]:
values_init = np.array([gamma, beta])
optimizationResult = minimize(get_expectation_value, values_init, options={'maxiter':500}, method='powell')
print(optimizationResult.fun)
print(optimizationResult.x)

In [ ]:
gamma_optimized = optimizationResult.x[0]
beta_optimized = optimizationResult.x[1]

qc = get_mainCircuit(num,gamma_optimized,beta_optimized)
backend = Aer.get_backend("qasm_simulator")
qc_transpiled = transpile(qc,backend)
result = backend.run(qc).result()
counts=result.get_counts()

commonLogic.my_plot_histogram(counts,gradation=True)